In [ ]:
import pandas as pd
import json
from tqdm import tqdm
import plotly.express as px
from pathlib import Path

def day_period(routes):
    index = pd.DatetimeIndex(routes['start_timestamp'])
    routes['day_period'] = -1
    routes.loc[index.indexer_between_time('00:00', '06:00'), 'day_period'] = 0
    routes.loc[index.indexer_between_time('06:00', '11:00'), 'day_period'] = 1
    routes.loc[index.indexer_between_time('11:00', '19:00'), 'day_period'] = 2
    routes.loc[index.indexer_between_time('19:00', '00:00'), 'day_period'] = 3
    return routes

def week_period(clear): # that's hell, i know
    clear['week_period'] = -1
    clear.loc[('2021-01-01' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2021-01-02'), 'week_period'] = 1
    clear.loc[('2020-12-31' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2021-01-01'), 'week_period'] = 1
    clear.loc[('2020-12-30' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-31'), 'week_period'] = 0
    clear.loc[('2020-12-29' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-30'), 'week_period'] = 0
    clear.loc[('2020-12-28' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-29'), 'week_period'] = 0
    clear.loc[('2020-12-27' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-28'), 'week_period'] = 1
    clear.loc[('2020-12-26' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-27'), 'week_period'] = 1
    clear.loc[('2020-12-25' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-26'), 'week_period'] = 0
    clear.loc[('2020-12-24' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-25'), 'week_period'] = 0
    clear.loc[('2020-12-23' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-24'), 'week_period'] = 0
    clear.loc[('2020-12-22' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-23'), 'week_period'] = 0
    clear.loc[('2020-12-21' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-22'), 'week_period'] = 0
    clear.loc[('2020-12-20' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-21'), 'week_period'] = 1
    clear.loc[('2020-12-19' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-20'), 'week_period'] = 1
    clear.loc[('2020-12-18' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-19'), 'week_period'] = 0
    clear.loc[('2020-12-17' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-18'), 'week_period'] = 0
    clear.loc[('2020-12-16' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-17'), 'week_period'] = 0
    clear.loc[('2020-12-15' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-16'), 'week_period'] = 0
    clear.loc[('2020-12-14' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-15'), 'week_period'] = 0
    clear.loc[('2020-12-13' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-14'), 'week_period'] = 1
    clear.loc[('2020-12-12' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-13'), 'week_period'] = 1
    clear.loc[('2020-12-11' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-12'), 'week_period'] = 0
    clear.loc[('2020-12-10' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-11'), 'week_period'] = 0
    clear.loc[('2020-12-09' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-10'), 'week_period'] = 0
    clear.loc[('2020-12-08' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-09'), 'week_period'] = 0
    clear.loc[('2020-12-07' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-08'), 'week_period'] = 0
    clear.loc[('2020-12-06' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-07'), 'week_period'] = 1
    clear.loc[('2020-12-05' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-06'), 'week_period'] = 1
    clear.loc[('2020-12-04' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-15'), 'week_period'] = 0
    clear.loc[('2020-12-03' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-04'), 'week_period'] = 0
    clear.loc[('2020-12-02' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-03'), 'week_period'] = 0
    clear.loc[('2020-12-01' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-02'), 'week_period'] = 0
    clear.loc[('2020-11-30' <= clear['start_timestamp']) & (clear['start_timestamp'] <= '2020-12-01'), 'week_period'] = 0
    return clear

def weather_period(clear, prop, values):
    clear[prop] = -1    
    
    night = pd.DatetimeIndex(clear['start_timestamp']).indexer_between_time('00:00', '11:00')
    day = pd.DatetimeIndex(clear['start_timestamp']).indexer_between_time('11:00', '19:00')
    evening = pd.DatetimeIndex(clear['start_timestamp']).indexer_between_time('19:00', '06:00')
    parts = [night, day, evening]
    v_index = 0
    dates = ['2020-12-01', '2020-12-02', '2020-12-03', '2020-12-04', '2020-12-05', '2020-12-06','2020-12-07', '2020-12-08', 
             '2020-12-09', '2020-12-10', '2020-12-11', '2020-12-12', '2020-12-13', '2020-12-14', '2020-12-15', '2020-12-16', '2020-12-17',
             '2020-12-18', '2020-12-19', '2020-12-20', '2020-12-21', '2020-12-22', '2020-12-23', '2020-12-24', '2020-12-25', '2020-12-26', 
             '2020-12-27', '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31'] # and this is too 
    for j in range(len(dates) - 1):
        for i in range(len(parts)):
            s = (dates[j] <= clear.loc[parts[i], 'start_timestamp']) & (clear.loc[parts[i], 'start_timestamp'] <= dates[j+1])
            clear.loc[(s)[s].index, prop] = values[v_index]
            if (i+2 != len(parts)):
                v_index += 1

    return clear

In [ ]:
routes = pd.read_csv("routes.csv")

In [ ]:
routes = week_period(day_period(routes)

In [ ]:
# Здесь не хватает скорости ветра, которую также можно достать из данных Гисметео

weather = pd.read_csv("weather.csv", sep = ";")

clouds_moscow__day = weather["cloud"].to_list()
snow_moscow_day = weather["weather"].to_list()
temp_moscow_day = weather["temp"].to_list()
wind_moscow_day = weather["windDir(from)"].to_list()
press_moscow_day = weather["pressure"].to_list() 

clouds_moscow_night = weather["cloud.1"].to_list()
snow_moscow_night = weather["weather.1"].to_list()
temp_moscow_night = weather["temp.1"].to_list()
wind_moscow_night = weather["windDir(from).1"].to_list()
press_moscow_night = weather["pressure.1"].to_list()

def weather_merger(day, night):
    result = []
    for i in range(len(day)):
        result.append(day[i])
        result.append(night[i])
    return result

clouds_moscow = weather_merger(clouds_moscow_day, clouds_moscow_night)
snow_moscow = weather_merger(snow_moscow_day, snow_moscow_night)
temp_moscow = weather_merger(temp_moscow_day, temp_moscow_night)
wind_moscow = weather_merger(wind_moscow_day, wind_moscow_night)
press_moscow = weather_merger(press_moscow_day, press_moscow_night)

props = ['clouds', 'snow', 'temperature', 'wind', 'pressure']
vals_moscow = [clouds_moscow, snow_moscow, temp_moscow, wind_moscow, press_moscow]

In [ ]:
for i in tqdm(range(len(props))):
    final = weather_period(final, props[i], vals_moscow[i])

In [ ]:
final.to_csv("final.csv")